In [1]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


#https://colab.research.google.com/drive/1N3LvAO0AXV4kBPbTMX866OwJM9YS6Ji2?usp=sharing#scrollTo=fl5W1gg5Jhzz

df_players = pd.read_csv("./../data_scrapped/atp_players.csv")
df_matchs = pd.read_csv("./../data_formatted/training_dgl_dataset.csv")

In [5]:
#On construit le graph

# lecture des données attendues
# tensor1[1] -> tensor2[1] 
# le player index en tensor1 a joué contre le player au meme index en tensor 2
# y stocke le résultat du match


tensor1=[]
tensor2=[]

for index,row in df_players.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    #print(len(winmatchs))
    if len(winmatchs) > 0:
        #print(row.player_id)
        for index2, row2 in winmatchs.iterrows():
                tensor1.append(index)
                tensor2.append(df_players.loc[df_players.player_id == row2.player2_id].index[0])

In [6]:
import numpy
import random

labels = numpy.ones(len(tensor1))

indexes = random.sample(range(0, len(tensor1)-1), ((len(tensor1)-1)//2))

for index in indexes:
    temp = tensor1[index]
    tensor1[index] = tensor2[index]
    tensor2[index] = temp
    labels[index] = 0

In [19]:
import torch_geometric.transforms as T
from torch_geometric.data import Data

# Create the heterogeneous graph data object:
#data = Data()

# Add the user nodes:
x = torch.Tensor(list(df_players[["birth_year","weight_kg","height_cm"]].values))  # [num_users, num_features_users]
x = torch.nan_to_num(x, nan=0.0)
#x = torch.masked_select(x, ~torch.isnan(x))
#x = torch.ones(df_players.shape[0])

edge_index = torch.stack([torch.tensor(tensor1), torch.tensor(tensor2)], dim=0)
#edge_attr = torch.Tensor(list(df_matchs[["player1_atprank","player1_oddsB365","player2_atprank","player2_oddsB365"]].values))
#y = torch.Tensor(list(df_matchs[["winner_player1"]].values))

labels = torch.Tensor(labels)

data = Data(x=x, edge_index=edge_index,y=labels) 
#data = T.ToUndirected()(data)
# Add the movie nodes:
#data['movie'].x = movie_features  # [num_movies, num_features_movies]

data

Data(x=[3446, 3], edge_index=[2, 51981], y=[51981])

In [24]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=False
)(data)
train_data, val_data, test_data

(Data(x=[3446, 3], edge_index=[2, 44184], y=[44184], edge_label=[44184], edge_label_index=[2, 44184]),
 Data(x=[3446, 3], edge_index=[2, 44184], y=[44184], edge_label=[5198], edge_label_index=[2, 5198]),
 Data(x=[3446, 3], edge_index=[2, 46783], y=[46783], edge_label=[10396], edge_label_index=[2, 10396]))

In [22]:
data.y

tensor([0., 1., 1.,  ..., 1., 0., 1.])

In [25]:
from torch_geometric.nn import GCNConv

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(3, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [26]:

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')


Epoch: 001, Loss: 1036127.6250, Val: 0.5516, Test: 0.5518
Epoch: 002, Loss: 268880.0312, Val: 0.5052, Test: 0.5061
Epoch: 003, Loss: 91835.0625, Val: 0.5023, Test: 0.5015
Epoch: 004, Loss: 167672.4375, Val: 0.5023, Test: 0.5015
Epoch: 005, Loss: 217630.3125, Val: 0.5023, Test: 0.5015
Epoch: 006, Loss: 192884.5156, Val: 0.5023, Test: 0.5015
Epoch: 007, Loss: 141116.1250, Val: 0.5029, Test: 0.5017
Epoch: 008, Loss: 97546.5547, Val: 0.5081, Test: 0.5092
Epoch: 009, Loss: 71924.5547, Val: 0.5127, Test: 0.5140
Epoch: 010, Loss: 57384.6484, Val: 0.5314, Test: 0.5321
Epoch: 011, Loss: 48634.3438, Val: 0.5516, Test: 0.5518
Epoch: 012, Loss: 41472.3477, Val: 0.5516, Test: 0.5518
Epoch: 013, Loss: 35145.1094, Val: 0.5516, Test: 0.5518
Epoch: 014, Loss: 30195.1191, Val: 0.5516, Test: 0.5518
Epoch: 015, Loss: 26385.5605, Val: 0.5518, Test: 0.5518
Epoch: 016, Loss: 24134.5625, Val: 0.5518, Test: 0.5518
Epoch: 017, Loss: 22563.0293, Val: 0.5518, Test: 0.5518
Epoch: 018, Loss: 21416.0703, Val: 0.5518

In [79]:
print(len(df_matchs[(df_matchs.player1_oddsB365 < df_matchs.player2_oddsB365) & (df_matchs.winner_player1 == 1)]))
print(len(df_matchs[(df_matchs.player1_oddsB365 > df_matchs.player2_oddsB365) & (df_matchs.winner_player1 == 0)]))
print(len(df_matchs))
print(16474 * 2 / 51981)

16474
16474
51981
0.6338469825513168


In [27]:
# Specify the edge you want to predict (e.g., edge from node 0 to node 1)
node_0 = 2000
node_1 = 3415

# Predict the direction of the edge
with torch.no_grad():
    model.eval()
    edge_label_index =  torch.tensor([[node_0, node_1], [node_1, node_0]], dtype=torch.long).t()
    print(edge_label_index)
    z = model.encode(data.x, data.edge_index)
    prediction = model.decode(z, edge_label_index)#.view(-1).sigmoid()
    print(prediction)

tensor([[2000, 3415],
        [3415, 2000]])
tensor([1180.3882, 1180.3882])
